In [1]:
import numpy as np
import pandas as pd
#import tensorflow.keras as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [2]:
image_size = 128
image_channels = 3

In [3]:
# Prepare dataset for training model

filenames=os.listdir("D:\datasets\kagglecatsanddogs_3367a\PetImages")

categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)

df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import pickle

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

model = Sequential()
model.add(Conv2D(32,(3,3), activation = 'relu', input_shape = (X.shape[1:])))
model.add(BatchNormalization(center = True, scale = True))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(BatchNormalization(center = True, scale = True))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3), activation = 'relu'))
model.add(BatchNormalization(center = True, scale = True))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization(center = True, scale = True))
model.add(Dropout(0.5))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#model.summary()



In [8]:
'''Keras Callback 是在调用fit时传入模型的一个对象，它在训练过程中的不同时间点都会被模型调用。它可以访问关于模型状态和性能的所有可用数据，
还可以采取行动：中断训练、保存模型、加载一组不同的权重或者改变模型的状态。也就是说，之前在训练模型的过程中，我们不知道模型的实时状态，
因此为了更好的监测和控制模型的训练过程，我们派出了一个特派员——回调函数，它可以根据情况记录、反馈或者采取措施。我们熟悉的训练进度条和fit
返回的history都是回调函数，只不过它俩因为太常用，所以被单独拎出来。'''

# Define callbacks

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callback_lists = []

callback_lists.append(EarlyStopping(monitor = 'acc', #监控模型的验证精度
                                    patience = 1)) #如果精度在多于一轮的时间（即两轮）内不再改善，就中断训练

callback_lists.append(ModelCheckpoint(filepath = 'my_model.h5', #目标文件的保存路径
                                      monitor = 'val_loss',  #监控验证损失
                                      save_best_only = True)) #只保存最佳模型

callback_lists.append(ReduceLROnPlateau(monitor = 'val_loss',  #监控模型的验证损失
                                      factor = 0.1,  #触发时将学习率乘以系数0.1
                                      patience = 10)) #若验证损失在10轮内都没有改善，则触发该回调函数

In [11]:
# Prepare data

df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15



# Training and validation data generator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        "D:\datasets\kagglecatsanddogs_3367a\PetImages\train",
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
       "D:\datasets\kagglecatsanddogs_3367a\PetImages\val",
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'D:\\datasets\\kagglecatsanddogs_3367a\\PetImages\train'